In [1]:
import numpy as np
import pandas as pd
pd.set_option('mode.copy_on_write', True)
import statsmodels.api as sm

In [2]:
rng = np.random.default_rng()

In [3]:
n = 100
p = 10
y = rng.normal(10, 2, size=n)  # Thing we're predicting.
X1 = np.ones((n, p))  # Regressors.  First column is for the intercept.
# Things we're predicting with
regressors = rng.normal(5, 1, size=(n, p-1)) 
# Put these into the design
# The regressors preceded by a column of ones to represent the intercept.
X1[:, 1:] = regressors

First design - no subtraction:

In [4]:
model1 = sm.OLS(y, X1)
fit1 = model1.fit()
fit1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.199
Model:                            OLS   Adj. R-squared:                  0.119
Method:                 Least Squares   F-statistic:                     2.490
Date:                Mon, 06 Mar 2023   Prob (F-statistic):             0.0137
Time:                        11:10:19   Log-Likelihood:                -214.11
No. Observations:                 100   AIC:                             448.2
Df Residuals:                      90   BIC:                             474.3
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.3396      3.475      0.961      0.339      -3.563      10.242
x1            -0.1602      0.211     -0.760      0.449      -0.579       0.258
x2            -0.4687      0.244     -1.919      0.058      -0.954       0.017
x3             0.5522      0.221      2.500      0.014       0.113       0.991
x4             0.0063      0.228      0.028      0.978      -0.447       0.460
x5             0.3289      0.206      1.600      0.113      -0.079       0.737
x6             0.5466      0.248      2.208      0.030       0.055       1.038
x7             0.0085      0.220      0.039      0.969      -0.429       0.446
x8            -0.0463      0.217     -0.214      0.831      -0.477       0.384
x9             0.5453      0.237      2.305      0.023       0.075       1.015
==============================================================================
Omnibus:                        1.227   Durbin-Watson:                   2.414
Prob(Omnibus):                  0.541   Jarque-Bera (JB):                1.285
Skew:                          -0.200   Prob(JB):                        0.526
Kurtosis:                       2.615   Cond. No.                         245.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Second model uses subtraction, but gives the same fitted values etc.  Notice we leave the first regressor in place, and replace the rest of the regressors with the differences.

In [5]:
X2 = X1.copy()
differences = np.diff(regressors, axis=1)
# Notice leaving column of ones and first regressor in place.
X2[:, 2:] = differences

In [6]:
model2 = sm.OLS(y, X2)
fit2 = model2.fit()
fit2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.199
Model:                            OLS   Adj. R-squared:                  0.119
Method:                 Least Squares   F-statistic:                     2.490
Date:                Mon, 06 Mar 2023   Prob (F-statistic):             0.0137
Time:                        11:10:19   Log-Likelihood:                -214.11
No. Observations:                 100   AIC:                             448.2
Df Residuals:                      90   BIC:                             474.3
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.3396      3.475      0.961      0.339      -3.563      10.242
x1             1.3126      0.683      1.922      0.058      -0.044       2.669
x2             1.4728      0.695      2.118      0.037       0.091       2.854
x3             1.9414      0.619      3.137      0.002       0.712       3.171
x4             1.3893      0.589      2.357      0.021       0.218       2.560
x5             1.3830      0.530      2.611      0.011       0.331       2.435
x6             1.0541      0.462      2.279      0.025       0.135       1.973
x7             0.5075      0.378      1.341      0.183      -0.244       1.259
x8             0.4990      0.321      1.556      0.123      -0.138       1.136
x9             0.5453      0.237      2.305      0.023       0.075       1.015
==============================================================================
Omnibus:                        1.227   Durbin-Watson:                   2.414
Prob(Omnibus):                  0.541   Jarque-Bera (JB):                1.285
Skew:                          -0.200   Prob(JB):                        0.526
Kurtosis:                       2.615   Cond. No.                         90.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Notice the R-squared and the F-statistic are exactly the same with the two models.  In fact, the predicted values are also the same (within the precision of the calculations):

In [7]:
predicted1 = fit1.predict()
predicted2 = fit2.predict()
np.allclose(predicted1, predicted2)

True

In [8]:
# Just for completeness - the maths behind the stuff above.
# Find the least-squares parameters with linear algebra "pinv".
params = np.linalg.pinv(X1) @ y
params

array([ 3.3395934 , -0.16022636, -0.46865428,  0.55218687,  0.0062739 ,
        0.32889603,  0.54658512,  0.00853995, -0.04629522,  0.54525186])

In [9]:
# This the same calculation that statsmodels did:
np.allclose(params, fit1.params)

True

In [10]:
# Predicted is just the parameters times the columns, added up.
predicted = np.sum(X1 * params, axis=1)
np.allclose(predicted, fit1.predict())

True

In [11]:
# Residuals are just the data minus the parameters.
residuals = y - predicted
sum_sq_resid = np.sum(residuals ** 2)
np.allclose(sum_sq_resid, fit1.ssr)

True